In [ ]:
from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）をセット
DATASET_SLUG = 'gns-codes'  # 変更する場合はここを更新
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 取得優先順位: 展開済み code/ があればそれを使う。無ければ code.zip を展開。どちらも無ければエラー。
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code ディレクトリも code.zip も見つかりません。Add Data で {DATASET_SLUG} を追加してください。")

if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
# unzip でサブディレクトリ1階層挟まった場合に平坦化
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children)==1 and children[0].is_dir():
        inner = children[0]
        for path in inner.iterdir():
            path.rename(repo_dir / path.name)
        inner.rmdir()
%cd $repo_dir
# 依存パッケージは Kaggle 環境の標準ライブラリを利用（追加インストールなし）


In [ ]:
!pip install torch-geometric 
# torch-scatter torch-sparse torch-cluster

In [ ]:
import torch

torch_ver = torch.__version__.split("+")[0]          # 例: "2.1.0+cu121" -> "2.1.0"
cuda_ver  = torch.version.cuda                      # 例: "12.1" / CPUなら None
cuda_tag  = "cpu" if cuda_ver is None else "cu" + cuda_ver.replace(".", "")

url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print("PyG wheels:", url)

# 必要なやつだけ入れる（radius_graphなら torch_cluster が本体）
!pip -q install torch-cluster -f {url}

# もし他も使うならまとめて
# !pip -q install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f {url}
# !pip -q install torch_geometric


In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'gns'
cfg.setdefault('method_options', {}).setdefault('gns', {})['edge_relative_velocity'] = True
cfg['amp_enable'] = False
cfg['ntraining_steps'] = 100
cfg['train_dataset_count'] = 50
cfg['valid_dataset_count'] = 10
cfg['validation_interval'] = 1000
cfg['max_grad_norm'] = 100
max_steps_cap = 70  # 学習・検証・ロールアウト評価の上限ステップ数
cfg['train_max_steps_per_trajectory'] = max_steps_cap
cfg['valid_max_steps_per_trajectory'] = max_steps_cap
cfg['rollout_max_steps'] = max_steps_cap

# Kaggle 入力データセットのパスに合わせて上書き
dataset_root = '/kaggle/input/dam-break-left-slosh2'
cfg['data_path'] = dataset_root
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code

import subprocess
import torch

gpu_count = torch.cuda.device_count()
if gpu_count >= 2:
    nproc = 2
    cmd = ["torchrun", f"--nproc_per_node={nproc}", "src/train.py", "--config", "config.yaml"]
elif gpu_count == 1:
    nproc = 1
    cmd = ["python", "src/train.py", "--config", "config.yaml"]
else:
    nproc = 0
    cmd = ["python", "src/train.py", "--config", "config.yaml"]
    print("GPU が検出できませんでした。CPU で実行します。")

print(f"検出GPU数: {gpu_count} -> 実行コマンド: {' '.join(cmd)}")
subprocess.run(cmd, check=True)


In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'gns'
cfg.setdefault('method_options', {}).setdefault('gns', {})['edge_relative_velocity'] = True
cfg['rollout_inference_max_examples'] = 1
max_steps_cap = 70  # 学習と同じ上限で推論ロールアウトを制限
cfg['train_max_steps_per_trajectory'] = max_steps_cap
cfg['valid_max_steps_per_trajectory'] = max_steps_cap
cfg['rollout_max_steps'] = max_steps_cap

# 推論時のデータセットパスを上書き
dataset_root = '/kaggle/input/dam-break-left-slosh2'
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code
!python src/train.py --config config_rollout.yaml
!python analyze_rollouts.py
!python visualize_rollout.py

In [ ]:
# dam_break_left valid 用の推論設定をノートブック内で組み立て（ファイル出力なし）
from pathlib import Path
import yaml
from train_config import Config

REPO = Path('/kaggle/working/code')
cfg_src = REPO / 'config_rollout.yaml'

dataset_root = REPO / 'datasets/out_fluid/dam_break_left'
model_file = REPO / 'models/model-10000 .pt'

if not model_file.exists():
    raise FileNotFoundError(f'モデルが見つかりません: {model_file}')
if not (dataset_root / 'valid.npz').exists():
    raise FileNotFoundError(f'valid.npz が見つかりません: {dataset_root}')

with cfg_src.open('r', encoding='utf-8') as f:
    cfg_dict = yaml.safe_load(f)

cfg_dict.update({
    'mode': 'rollout',
    'scenario': 'fluid',
    'data_path': str(dataset_root),
    'model_path': str(REPO / 'models'),
    'model_file': str(model_file),
    'rollout_dataset': 'valid',
    'rollout_inference_max_examples': None,  # valid 全件
    'rollout_max_steps': None,
    'train_max_steps_per_trajectory': None,
    'valid_max_steps_per_trajectory': None,
})
fluid_opts = cfg_dict.setdefault('scenario_options', {}).setdefault('fluid', {})
fluid_opts['dataset'] = str(dataset_root)
fluid_opts['rollout_dataset'] = 'valid'
fluid_opts.setdefault('description', 'dam_break_left valid dataset')

cfg = Config(**cfg_dict)
print('モデル:', model_file)
print('データセット:', dataset_root)
print('cfg.mode:', cfg.mode, '| rollout_dataset:', cfg.rollout_dataset)

In [ ]:
# 1-step 誤差を時刻ごとに計算
import numpy as np
import torch

import data_loader
import reading_utils
from rollout_utils import rollout
from simulator_factory import _get_simulator
from train import _prepare_scenario
from train_config import INPUT_SEQUENCE_LENGTH
from train_paths import _prepare_model_directory, _resolve_model_run_directory, _resolve_model_path
from train_utils import _resolve_rollout_dataset_path

# 上のセルで組み立てた cfg をそのまま利用
cfg = _prepare_scenario(cfg)
_prepare_model_directory(cfg)
_resolve_model_run_directory(cfg)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
metadata_key = cfg.active_scenario.rollout_metadata_split or 'rollout'
metadata = reading_utils.read_metadata(cfg.data_path, metadata_key)

simulator = _get_simulator(metadata, cfg.noise_std, cfg.noise_std, device, cfg)
model_path = _resolve_model_path(cfg)
print('モデルをロード:', model_path)
simulator.load(model_path)
simulator.to(device)
simulator.eval()

dataset_path = _resolve_rollout_dataset_path(cfg)
print('データセット:', dataset_path)
loader = data_loader.get_data_loader_by_trajectories(dataset_path)

max_eval_examples = None  # 例: 5 にすると先頭5軌道だけ評価
max_steps_cap = None      # 例: 50 にすると 50 ステップまでに制限

rmse_time_series = []
dt = float(metadata.get('dt', 1.0))

with torch.no_grad():
    for ex_idx, features in enumerate(loader):
        if max_eval_examples is not None and ex_idx >= int(max_eval_examples):
            break

        positions = features[0].to(device)
        particle_type = features[1].to(device)
        if len(features) == 4:
            material_property = features[2].to(device)
            n_particles = torch.tensor([int(features[3])], dtype=torch.int32, device=device)
        else:
            material_property = None
            n_particles = torch.tensor([int(features[2])], dtype=torch.int32, device=device)

        nsteps = positions.shape[1] - INPUT_SEQUENCE_LENGTH
        if max_steps_cap is not None:
            nsteps = min(nsteps, int(max_steps_cap))

        _, loss = rollout(
            simulator,
            positions,
            particle_type,
            material_property,
            n_particles,
            nsteps,
            device,
            show_progress=False,
        )

        rmse_per_step = torch.sqrt(loss.mean(dim=(1, 2))).cpu().numpy()
        time_axis = dt * (INPUT_SEQUENCE_LENGTH + np.arange(1, len(rmse_per_step) + 1))
        rmse_time_series.append({
            'example': ex_idx,
            'time': time_axis,
            'rmse': rmse_per_step,
        })

print(f'評価完了: {len(rmse_time_series)} 件')

In [ ]:
# 時刻ごとの RMSE を可視化（各軌道 + 平均）
import numpy as np
import matplotlib.pyplot as plt

if not rmse_time_series:
    raise RuntimeError('rmse_time_series が空です。前のセルを確認してください。')

max_len = max(len(entry['rmse']) for entry in rmse_time_series)
time_grid = np.full((len(rmse_time_series), max_len), np.nan, dtype=float)
rmse_grid = np.full_like(time_grid, np.nan)

plt.figure(figsize=(8, 5))
for i, entry in enumerate(rmse_time_series):
    l = len(entry['rmse'])
    time_grid[i, :l] = entry['time']
    rmse_grid[i, :l] = entry['rmse']
    plt.plot(entry['time'], entry['rmse'], alpha=0.25, label=f"ex{entry['example']}")

mean_time = np.nanmean(time_grid, axis=0)
mean_rmse = np.nanmean(rmse_grid, axis=0)
plt.plot(mean_time, mean_rmse, color='red', lw=2, label='mean')

plt.xlabel('time [s]')
plt.ylabel('position RMSE (1-step)')
plt.title('dam_break_left valid: 1-step error vs. time')
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
